In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import glob
from glob import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from PIL import Image
# for progress bar
from tqdm import tqdm_notebook, tqdm, trange
#to print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

# from pytorch_pretrained_vit import ViT

# from linformer import Linformer
# from torch.utils.tensorboard import SummaryWriter

In [2]:
source_PATH = "/home/tzortzis/Data/IRI-CATs-light/real_data/short_tasks/source_robotics/"
domain_PATH_train = "/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/train/"
domain_PATH_test = "/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/"
EXT = "*.csv"
img_EXT = "*.jpg"

In [3]:
def get_frame(PATH):
    all_csv_files = []
    all_img_files = []
    for path, subdir, files in os.walk(PATH):
        for file in glob(os.path.join(path, EXT)):
            if file.endswith('data.csv'):
                all_csv_files.append(file)
        for i in glob(os.path.join(path, img_EXT)):
            all_img_files.append(i)


    li = []

    for filename in all_csv_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)



    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame, all_img_files, all_csv_files

In [4]:
source_frame, all_img_files_source, all_csv_files_source = get_frame(source_PATH)
domain_train_frame,  all_img_files_domain_train, all_csv_files_domain_train = get_frame(domain_PATH_train)
domain_test_frame,  all_img_files_domain_test, all_csv_files_domain_test = get_frame(domain_PATH_test)

In [5]:
states = source_frame.cloth_state.unique()
def get_idx(frame, states):
    state = states
    state2idx = dict((state,idx) for idx, state in enumerate(state))
    idx2state = dict((idx,state) for idx,state in enumerate(state))
    frame['label_idx'] = [state2idx[b] for b in frame.cloth_state]
    return

In [6]:
get_idx(source_frame, states)
get_idx(domain_train_frame, states)
get_idx(domain_test_frame, states)

In [7]:
domain_test_frame.cloth_state.unique()
# domain_test_frame.label_idx.unique()

array(['lifted', 'semi-lifted-twohands', 'folded'], dtype=object)

In [8]:
class CustomDataset(Dataset):
    def __init__(self, labels_df, img_path, transform=None):
        self.labels_df = labels_df
        self.img_path = img_path
        self.transform = transform
        
        
    def __len__(self):
        return self.labels_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = self.labels_df.filename[idx]
        for file in self.img_path:
            if file.endswith(image_name + '.jpg'):
                img = Image.open(file)
                filename = file
#                 print(file)
        label = self.labels_df.label_idx[idx]
#         print(file) 
            
        if self.transform:
            img = self.transform(img)

        return img, label, filename

In [9]:
input_size = 224
bs = 1

data_transforms = {
    'train': transforms.Compose([
    transforms.Resize(input_size),
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#these vectors is the mean and the std from the statistics in imagenet. They are always the same as far as I can recall
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# source_dataset = CustomDataset(source_frame, all_img_files_source)
# target_train =  CustomDataset(domain_train_frame, all_img_files_domain_train)
# target_test =  CustomDataset(domain_test_frame, all_img_files_domain_test)

In [10]:
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
train_split_idx, val_split_idx = next(iter(stratified_split.split(source_frame.filename, source_frame.cloth_state)))
train_df = source_frame.iloc[train_split_idx].reset_index()
val_df = source_frame.iloc[val_split_idx].reset_index()

In [11]:
train_df.cloth_state.unique()
train_df.label_idx.unique()

array(['flat', 'semi-lifted-twohands', 'lifted', 'folded'], dtype=object)

array([0, 1, 3, 2])

In [12]:
source_train_dataset = CustomDataset(train_df, all_img_files_source, transform = data_transforms['train'])
val_dataset = CustomDataset(val_df, all_img_files_source, transform = data_transforms['val'])
target_train_dataset = CustomDataset(domain_train_frame, all_img_files_domain_train, transform = data_transforms['train'])
test_dataset = CustomDataset(domain_test_frame, all_img_files_domain_test, transform = data_transforms['val'])

In [13]:
train_source_loader = DataLoader(source_train_dataset, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
train_target_loader = DataLoader(target_train_dataset, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

In [14]:
domain_test_frame.head()

,filename,cloth_state,false_labeling,label_idx
0,robotic_no_release_orange_0001,lifted,NaN,3
1,robotic_no_release_orange_0002,lifted,NaN,3
2,robotic_no_release_orange_0003,lifted,NaN,3
3,robotic_no_release_orange_0004,lifted,NaN,3
4,robotic_no_release_orange_0005,lifted,NaN,3


In [15]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

In [17]:
p = figure(
    width=1000,
    height=1000,
    title="Task Stream",
    toolbar_location="above",
    x_axis_type="datetime",
)
labs = ['lifted', 'semi-lifted-twohands', 'flat', 'folded']
p.rect(
    source=ColumnDataSource(domain_test_frame),
    x="index",
    y="label_idx",
    width=0.01,
    height=0.4
)

show(p)

GlyphRenderer(id='1075', ...)

In [21]:
for data in test_loader:
    inps, labels, filename = data
    filename, labels
    

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_0001.jpg',),
 tensor([3]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_0002.jpg',),
 tensor([3]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_0003.jpg',),
 tensor([3]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_0004.jpg',),
 tensor([3]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_0005.jpg',),
 tensor([3]))

...

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_1409.jpg',),
 tensor([2]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_1410.jpg',),
 tensor([2]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_1411.jpg',),
 tensor([2]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_1412.jpg',),
 tensor([2]))

(('/home/tzortzis/Data/IRI-CATs-light/real_data/long_tasks/cropped_robotic/test/orange/no_release_orange/robotic_no_release_orange_1413.jpg',),
 tensor([2]))